In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

In [2]:
batch_size = 100
learning_rate = 1e-3
num_epoches = 20

In [3]:
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
for i, data in enumerate(train_loader, 2):
        img, label = data

In [4]:
class Rnn(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_class):
        super(Rnn, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layer, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, n_class)

    def forward(self, x):
        # h0 = Variable(torch.zeros(self.n_layer, x.size(1),
        #   self.hidden_dim)).cuda()
        # c0 = Variable(torch.zeros(self.n_layer, x.size(1),
        #   self.hidden_dim)).cuda()
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.classifier(out)
        return out

In [5]:
model = Rnn(28, 128, 2, 10)  
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        b, c, h, w = img.size()
        assert c == 1, 'channel must be 1'
        img = img.squeeze(1)
        # img = img.view(b*h, w)
        # img = torch.transpose(img, 1, 0)
        # img = img.contiguous().view(w, b, -1)

        img = Variable(img).cuda()
        label = Variable(label).cuda()

        # foward
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        b, c, h, w = img.size()
        assert c == 1, 'channel must be 1'
        img = img.squeeze(1)
        # img = img.view(b*h, w)
        # img = torch.transpose(img, 1, 0)
        # img = img.contiguous().view(w, b, h)

        img = Variable(img, volatile=True).cuda()
        label = Variable(label, volatile=True).cuda()

        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))
    print()

# save
torch.save(model.state_dict(), './rnn.pth')

epoch 1
**********
[1/20] Loss: 0.138168, Acc: 0.958700
[1/20] Loss: 0.122203, Acc: 0.963750
Finish 1 epoch, Loss: 0.122203, Acc: 0.963750
Test Loss: 0.085203, Acc: 0.973700

epoch 2
**********
[2/20] Loss: 0.081565, Acc: 0.975233
[2/20] Loss: 0.077411, Acc: 0.976500
Finish 2 epoch, Loss: 0.077411, Acc: 0.976500
Test Loss: 0.061919, Acc: 0.980700

epoch 3
**********
[3/20] Loss: 0.061020, Acc: 0.980967
[3/20] Loss: 0.061495, Acc: 0.981117
Finish 3 epoch, Loss: 0.061495, Acc: 0.981117
Test Loss: 0.068692, Acc: 0.980500

epoch 4
**********
[4/20] Loss: 0.048232, Acc: 0.985000
[4/20] Loss: 0.050649, Acc: 0.984417
Finish 4 epoch, Loss: 0.050649, Acc: 0.984417
Test Loss: 0.050845, Acc: 0.984600

epoch 5
**********
[5/20] Loss: 0.036948, Acc: 0.988900
[5/20] Loss: 0.038038, Acc: 0.988417
Finish 5 epoch, Loss: 0.038038, Acc: 0.988417
Test Loss: 0.044629, Acc: 0.986700

epoch 6
**********
[6/20] Loss: 0.032861, Acc: 0.990567
[6/20] Loss: 0.036233, Acc: 0.989400
Finish 6 epoch, Loss: 0.036233, 